In [1]:
# Bibliotecas utlizadas
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Dataset de avaliações de filmes
avaliacoes = pd.read_csv('ml-100k/u.data', sep='\t', names=['id_usuario', 'id_filme', 'avaliacao', 'timestamp'])

In [3]:
# Dataset dos dados dos filmes
filmes = pd.read_csv('ml-100k/u.item', sep='|', encoding='ISO-8859-1', header=None, usecols=[0, 1], names=['id_filme', 'titulo_filme'])


In [11]:
# Mesclar os dois dataframes para incluir os títulos dos filmes
dados = pd.merge(avaliacoes, filmes, on='id_filme')


In [12]:
# Matriz (usuário por filme)
matriz_usuario_filme = dados.pivot_table(index='id_usuario', columns='titulo_filme', values='avaliacao', aggfunc='mean').fillna(0)

In [14]:
# SVD (reduz dimensionalidade da matriz)
svd = TruncatedSVD(n_components=50)  # 50 componentes
matriz_reduzida = svd.fit_transform(matriz_usuario_filme)

In [15]:
# Similaridade entre usuários usando a similaridade do cosseno
similaridade_usuarios = cosine_similarity(matriz_reduzida)

In [16]:
# Função para recomendar filmes
def recomendar_filmes(indice_usuario, n_recomendacoes=5):
    # Encontrar os usuários mais semelhantes ao usuário fornecido
    usuarios_semelhantes = similaridade_usuarios[indice_usuario].argsort()[::-1][1:]  
    
    # Filmes recomendados com base em usuários semelhantes
    filmes_recomendados = []
    for usuario_semelhante in usuarios_semelhantes:
        # Filmes que o usuário semelhante assistiu e o usuário atual não assistiu
        filmes_assistidos_usuario = matriz_usuario_filme.columns[matriz_usuario_filme.iloc[usuario_semelhante] > 0]
        filmes_assistidos_alvo = matriz_usuario_filme.columns[matriz_usuario_filme.iloc[indice_usuario] > 0]
        novas_recomendacoes = set(filmes_assistidos_usuario) - set(filmes_assistidos_alvo)
        
        filmes_recomendados.extend(novas_recomendacoes)
        if len(filmes_recomendados) >= n_recomendacoes:
            break
    
    return list(filmes_recomendados)[:n_recomendacoes]

In [19]:
# Teste (usuário 1)
indice_usuario = 0  # usuário 1
recomendacoes = recomendar_filmes(indice_usuario, n_recomendacoes=5)

print(f"Recomendações de filmes para o usuário {indice_usuario + 1}: {recomendacoes}")


Recomendações de filmes para o usuário 32: ['Toy Story (1995)', 'Bound (1996)', 'Johnny 100 Pesos (1993)', 'One Fine Day (1996)', 'Four Rooms (1995)']
